# Marketing Data Foundation Setup Script

## Project Setup using Notebook

### *Pre-requisites:*

1. Role with access to create Compute Pools, create Warehouses and Databases
2. Any environment to run notebooks like VS code, Jupyterlab, etc.
3. Have Docker Desktop installed
4. Install and configure the Snow CLI to deploy the application in your account.


### Config files
The proyect contains some config file that should be updated with your own environment variables 
1. [app.config.json](app.config.json)

## Setup local environment

In [ ]:
! python3.9 -m venv .venv 
! source .venv/bin/activate 
! pip install -r requirements.txt

### Load App Configurations

In [ ]:
from scripts.app_config import get_app_config

app_conf_file = 'app.config.json'
app_config = get_app_config(app_conf_file)

print(f'Success')

## Create Role


In [ ]:
from scripts.auth import get_conn_obj
from scripts.executeStatement import executeStatement
import os

user = os.environ['SNOWFLAKE_USER']
role = f"{app_config['database']}_ROLE"
print(executeStatement(f"CREATE ROLE IF NOT EXISTS {role};"))
print(executeStatement(f"GRANT CREATE COMPUTE POOL ON ACCOUNT TO ROLE {role};"))
print(executeStatement(f"GRANT BIND SERVICE ENDPOINT ON ACCOUNT TO ROLE {role};"))
print(executeStatement(f"GRANT CREATE WAREHOUSE ON ACCOUNT TO ROLE {role};"))
print('Success')


### Replace application configuration files keys

In [ ]:
import os

from scripts.update_file_variables import file_replace

app_config_f = 'app/src/manifest.yml'
fullstack_config_f = 'app/src/fullstack.yaml'
makefile_f = 'Makefile'

replace_map = {
    "<image_repository_path>": os.environ["IMAGE_REPOSITORY"],
    "<image_repo_short_path>": os.environ["IMAGE_REPO_SHORT"],
    "<role>": role
}

file_replace(app_config_f, replace_map)
file_replace(fullstack_config_f, replace_map)
file_replace(makefile_f, replace_map)
print(f'Success')

# Clear resources (optional)

In [ ]:
! cd app && snow app teardown

In [ ]:
# print(sp_session.sql(f"USE DATABASE {app_config['database']}"))
from scripts.executeStatement import executeStatement

print(executeStatement(f"ALTER COMPUTE POOL IF EXISTS {app_config['compute_pool']} STOP ALL;"))
print(executeStatement(f"DROP SERVICE IF EXISTS {app_config['container_service']}"))
print(executeStatement(f"DROP COMPUTE POOL IF EXISTS {app_config['compute_pool']}"))
print(executeStatement(f"DROP IMAGE REPOSITORY IF EXISTS {app_config['image_stage']};"))
print(executeStatement(f"DROP DATABASE IF EXISTS {app_config['database']};"))
print(executeStatement(f"DROP WAREHOUSE IF EXISTS {app_config['warehouse']};"))


## Setup Snowflake

### Create database, schema and stages

In [ ]:
create_app_package = f"CREATE DATABASE IF NOT EXISTS {app_config['database']};"
create_dedicated_warehouse = f"CREATE WAREHOUSE IF NOT EXISTS {app_config['warehouse']} WITH WAREHOUSE_SIZE = 'XSMALL' AUTO_SUSPEND = 60 AUTO_RESUME = TRUE INITIALLY_SUSPENDED = TRUE;"
grant_database = f"GRANT ALL PRIVILEGES ON DATABASE {app_config['database']} TO ROLE {role}"



create_app_schema = f"CREATE SCHEMA IF NOT EXISTS {app_config['database']}.{app_config['schema']};"

grant_schema = f"GRANT ALL PRIVILEGES ON SCHEMA {app_config['database']}.{app_config['schema']} TO ROLE {role};"

grant_tables = f"GRANT SELECT ON ALL TABLES IN SCHEMA {app_config['database']}.{app_config['schema']} TO ROLE {role};"

create_image_repo = f"CREATE IMAGE REPOSITORY IF NOT EXISTS {app_config['database']}.{app_config['schema']}.{app_config['image_stage']};"

print(executeStatement(create_app_package))
print(executeStatement(create_app_schema))
print(executeStatement(create_image_repo))
print(executeStatement(grant_database))
print(executeStatement(grant_schema))
print(executeStatement(grant_tables))
print(executeStatement(create_dedicated_warehouse))

### Upload Sample Data

In [ ]:
create_sample_db = f"CREATE DATABASE IF NOT EXISTS {app_config['sample_db']};"
create_sample_schema = f"CREATE SCHEMA IF NOT EXISTS {app_config['sample_db']}.{app_config['sample_schema']};"
create_sample_stage = f"CREATE STAGE IF NOT EXISTS {app_config['sample_db']}.{app_config['sample_schema']}.{app_config['sample_stage']};"

print(executeStatement(create_sample_db))
print(executeStatement(create_sample_schema))
print(executeStatement(create_sample_stage))

In [ ]:
from scripts.executeStatement import executeCopyToStage
sample_stage= f"@{app_config['sample_db']}.{app_config['sample_schema']}.{app_config['sample_stage']}"
ga_data = "data/ga_data/"
sf_data = "data/sf_data/"
worldcities = "data/worldcities.csv"

print(executeCopyToStage(ga_data,f"{sample_stage}/data/ga_data/"))
print(executeCopyToStage(sf_data, f"{sample_stage}/data/sf_data/"))
print(executeCopyToStage(worldcities, f"{sample_stage}/data"))

In [ ]:
from scripts.update_file_variables import file_replace

data_script_f = 'scripts/build_raw_samples.sql'

replace_map = {
    "<DB>": app_config['sample_db'],
    "<SCHEMA>": app_config['sample_schema'],
    "<STAGE>": app_config['sample_stage']
}

file_replace(data_script_f, replace_map)
print(f'Success')

In [ ]:
! snow sql -f scripts/build_raw_samples.sql

### Build and Upload Docker Images

In [ ]:
! make all

### Deploy Native Application

In [ ]:
! cd app && snow app run

### Create Compute Pool

In [ ]:
import os
import subprocess
appDeployed = subprocess.run (['snow', 'app', 'version', 'list'], stdout=subprocess.PIPE, cwd='app')
result = appDeployed.stdout.decode("utf-8")
print(result)
appPackageName = result[result.index("Marketing_Data_Foundation"): result.index("\n")]
appName = appPackageName.replace("_pkg_","_")
print(appName)

### Load Predefined Data Models

In [ ]:
from scripts.load_models import load_models

models_folder = "backend/predefined_models"

load_models(appName, models_folder)

In [ ]:
from scripts.executeStatement import executeStatement
create_compute_pool_sql = f"CREATE COMPUTE POOL IF NOT EXISTS {app_config['compute_pool']} for application {appName}\
    MIN_NODES = 1 \
    MAX_NODES = 1 \
    AUTO_SUSPEND_SECS = 120 \
    INSTANCE_FAMILY = CPU_X64_S;"
grant_usage_sql = f"GRANT USAGE, MONITOR ON COMPUTE POOL {app_config['compute_pool']} \
    TO application {appName};"

grant_usage_wh = f"GRANT USAGE, MONITOR ON WAREHOUSE {app_config['warehouse']} \
    TO application {appName};"

grant_bind_service = f"GRANT BIND SERVICE ENDPOINT ON ACCOUNT TO application {appName};"

print(executeStatement(create_compute_pool_sql))
print(executeStatement(grant_usage_sql))
print(executeStatement(grant_usage_wh))
print(executeStatement(grant_bind_service))



#### Upload Semantic Model Configuration

In [ ]:
from scripts.upload_files import upload_files_stage
from scripts.update_file_variables import file_replace

database = appName
schema = 'LLM'
semantic_models_stage = 'SEMANTIC_MODEL'
llm_config_stage = 'CONFIGURATION'

replace_map = {
    "<target_database>": appName
}
file_replace('./assistant/semantic_models/UnifiedMarketingModel_CAMPAIGN_PERF.yaml', replace_map)

subprocess.run(['snow', 'stage', 'copy', './assistant/config/assistant_config.yaml', f"@{database}.{schema}.{llm_config_stage}"])
subprocess.run(['snow', 'stage', 'copy', './assistant/semantic_models/UnifiedMarketingModel_CAMPAIGN_PERF.yaml', f"@{database}.{schema}.{semantic_models_stage}"])

print('Success')

In [ ]:
grant_sample_db = f"GRANT USAGE ON DATABASE {app_config['sample_db']} TO APPLICATION {appName};"
grant_sample_schema = f"GRANT USAGE ON SCHEMA {app_config['sample_db']}.{app_config['sample_schema']} TO APPLICATION {appName};"
grant_sample_select = f"GRANT SELECT ON ALL TABLES IN SCHEMA {app_config['sample_db']}.{app_config['sample_schema']} TO APPLICATION  {appName};"

print(executeStatement(grant_sample_db))
print(executeStatement(grant_sample_schema))
print(executeStatement(grant_sample_select))


## Start container service

In [ ]:
service_query = f"call {appName}.app_public.start_app(\'{app_config['compute_pool']}\',\'{app_config['warehouse']}\')"
executeStatement(service_query)

### Show container endpoint

In [ ]:
from scripts.executeStatement import executeStatement
from scripts.endpoint_provider import get_public_url_na
get_public_url_na(appName, executeStatement)